In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from dataclasses import dataclass, field
from abc import ABC, abstractmethod
from enum import Enum, IntEnum, auto
from collections.abc import Iterable
from typing import Mapping, Any, Sequence, Optional

import matplotlib.pyplot as plt

from numpy.typing import NDArray

In [2]:
class Instrument(Enum):
    STOCK = auto()
    CALL = auto()
    PUT = auto()


class Side(IntEnum):
    LONG = 1
    SHORT = -1


In [3]:
ArrayF = NDArray[np.float64]

In [4]:
class Currency(Enum):
    USD = auto()

In [ ]:
@dataclass(frozen=True, slots=True)
class AssetID:
    symbol: str
    exchange: Optional[str] = None
    currency: Currency = Currency.USD

    def __post_init__(self) -> None:

        sym = self.symbol.strip().lower()
        exc = self.exchange.strip().lower()
        cur = self.currency.strip().lower()

        if not sym:
            raise ValueError("Symbol cannot be empty")
        if (len(cur) != 3 or not cur.isalpha()):
            raise ValueError(f"Currency must be three letters and alpha numeric like 'USD'. Got {cur}")

        object.__setattr__(self, "symbol", sym)
        object.__setattr__(self, "exchange", exc)
        object.__setattr__(self, "currency", cur)

In [ ]:
@dataclass(frozen=True, slots=True) 
class GroupID:
    instrument: Instrument
    symbol: str

    def __post_init__(self) -> None:
        sym = self.symbol.strip().lower()

        if not sym:
            raise ValueError("Symbol cannot be empty")
        if not self.instrumenttnt:
            raise ValueError("Instrument cannot be empty")

        object.__setattr__(self, "symbol", sym)

In [5]:
@dataclass(frozen=True)
class Leg(ABC):
    side: Side
    quantity: int

    @abstractmethod
    def pnl(self, S: float) -> float:
        ...

    def group_id(self) -> GroupID


@dataclass(frozen=True)
class CallLeg(Leg):
    strike_price: float
    premium: float

    def pnl(self, S: ArrayF) -> ArrayF:
        sign = int(self.side)
        intrinsic = np.maximum(S - self.strike_price, 0.0)
        payoff = sign * (intrinsic - self.quantity * self.premium)
        return self.quantity * 100 * payoff


@dataclass(frozen=True)
class PutLeg(Leg):
    strike_price: float
    premium: float

    def pnl(self, S: ArrayF) -> ArrayF:
        sign = int(self.side)
        intrinsic = np.maximum(self.strike_price - S, 0.0)
        payoff = sign * (intrinsic - self.quantity * self.premium)
        return self.quantity * 100 * payoff


@dataclass(frozen=True)
class StockLeg(Leg):
    initial_price: float

    def pnl(self, S: ArrayF) -> ArrayF:
        return int(self.side) * self.quantity * (S - self.initial_price)

In [6]:
def leg_from_dict(d: Mapping[str, Any]) -> Leg:
    try:
        t = d["Type"]
        side = d["Side"]
        qty = int(d["Quantity"])
    except KeyError as e:
        raise ValueError(f"Missing required key: {e.args[0]}") from None

    if t == Instrument.STOCK:
        if "Initial_price" not in d:
            raise ValueError("STOCK requires Initial_price")
        return StockLeg(side=side,
                        quantity=qty,
                        initial_price=float(d["Initial_price"]))
    if t == Instrument.CALL:
        if "Strike_price" not in d:
            raise ValueError("CALL requires Strike_price")
        elif "Premium" not in d:
            raise ValueError("Call requires Premium")
        return CallLeg(side=side,
                       quantity=qty,
                       strike_price=float(d["Strike_price"]),
                       premium=float(d["Premium"]))
    if t == Instrument.PUT:
        if "Strike_price" not in d:
            raise ValueError("PUT requires Strike_price")
        elif "Premium" not in d:
            raise ValueError("Call requires Premium")
        return PutLeg(side=side,
                      quantity=qty,
                      strike_price=float(d["Strike_price"]),
                      premium=float(d["Premium"]))
    raise ValueError(f"Unknown Type: {t!r}")

In [7]:
class Position():
    def __init__(self, legs: tuple[Leg]):
        self.strike_prices = [leg.strike_prices for leg in legs]
    
    @classmethod
    def from_spec(cls, specs: Iterable[Mapping[str, Any]]):
        return cls(legs=tuple(leg_from_dict(s) for s in specs))

    def pnl_range(self, start: float, stop: float, num_prices: int = 50) -> ArrayF:
        underlying_price_range = np.linspace(start, stop, num=num_prices)
        pl_range = np.zeros_like(underlying_price_range)
        for leg in self.legs:
            pl_range += leg.pnl(underlying_price_range)

        return (underlying_price_range, pl_range)

    def maximum_loss():
        pass

    def maximum_gain():
        pass

In [8]:
specs = [
    {
        "Type": Instrument.CALL,
        "Strike_price": 275,
        "Side": Side.SHORT,
        "Quantity": 1,
        "Premium": 7.50,
        "Underlying":
    },
    {
        "Type": Instrument.STOCK,
        "Side": Side.LONG,
        "Quantity": 1,
        "Initial_price": 250,
    }
]

pos = Position.from_spec(specs)
x, y = pos.pnl_range(235, 150)

In [9]:
def graph_pnl(x: Sequence[float], y: Sequence[float]):
    fig, ax = plt.subplots()
    ax.plot(x, y)
    ax.axhline(0, color="k")
    ax.axvline(underlying_price_range[np.argmin(abs(payoff))], color="k")
    ax.set(xlabel="Asset Price", ylabel="P/L", title="P/L for Position")
    plt.show()

In [91]:
underlying_price_range = np.linspace(235, 310, 100)
pl_range = np.zeros_like(underlying_price_range)
call_short = leg_from_dict(specs[0])
sign = int(call_short.side)


intrinsic = np.maximum(underlying_price_range - call_short.strike_price, 0.0)
payoff = sign * (intrinsic - call_short.quantity * call_short.premium)
payoff = call_short.quantity * 100 * payoff
payoff

array([-750.        , -750.        , -750.        , -750.        ,
       -750.        , -750.        , -750.        , -750.        ,
       -750.        , -750.        , -750.        , -750.        ,
       -750.        , -750.        , -750.        , -750.        ,
       -750.        , -750.        , -750.        , -750.        ,
       -750.        , -750.        , -750.        , -750.        ,
       -750.        , -750.        , -750.        , -750.        ,
       -750.        , -750.        , -750.        , -750.        ,
       -750.        , -750.        , -750.        , -750.        ,
       -750.        , -750.        , -750.        , -750.        ,
       -750.        , -750.        , -750.        , -750.        ,
       -750.        , -750.        , -750.        , -750.        ,
       -750.        , -750.        , -750.        , -750.        ,
       -750.        , -734.84848485, -659.09090909, -583.33333333,
       -507.57575758, -431.81818182, -356.06060606, -280.30303

In [10]:
graph_pnl(underlying_price_range, payoff)

NameError: name 'underlying_price_range' is not defined

np.float64(282.72727272727275)